## Run Inference on all deployed endpoints: Various combinations of payloads, concurrency levels, model configurations
---------------------
*This notebook works best with the conda_python3 kernel on a ml.t3.medium machine*.

#### This step of our solution design includes running inferences on all deployed model endpoints (with different configurations, concurrency levels and payload sizes). This notebook runs inferences in a manner that is calls endpoints concurrently and asychronously to generate responses and record metrics. Here are some of the key components:

- **Accessing the deployed endpoints**, creating a predictor object for these endpoints to call them during inference time.

- **Functions to define metrics**: This notebook sets stage for metrics to be recorded during the time of invocation of all these models for benchmarking purposes.

- **Running Actual Inferences**: Once the metrics are defined, we set a blocker function that is responsible for creating inference on a single payload called get_inference. We then run a series of asynchronous functions that can be viewed in the code (link above), to create asychronous inferefences on the deployed models. The way we send requests are by creating combinations: this means creating combinations of payloads of different sizes that can be viewed in the config.yml file, with different concurrency levels (in this case we first go through all patches of payloads with a concurrency level of 1, then 2, and then 4). You can set this to your desired value.

In [60]:
## auto reload all of the changes made in the config/globals.py file 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Import all of the necessary libraries below to run this notebook

In [61]:
import glob
import time
import json
import copy
import boto3
import asyncio
import logging
import itertools
import sagemaker
import pandas as pd
from globals import *
from datetime import datetime
from transformers import AutoTokenizer
from sagemaker.predictor import Predictor
from utils import load_config, count_tokens
from sagemaker.serializers import JSONSerializer
from typing import Dict, List, Optional, Tuple, Union

#### Pygmentize globals.py to view and use any of the globally initialized variables 

In [62]:
# global constants
!pygmentize globals.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


import os
import yaml
from enum import Enum
from pathlib import Path

CONFIG_FILEPATH_FILE: str = "config_filepath.txt"

CONFIG_FILE: str = Path(CONFIG_FILEPATH_FILE).read_text()
print(f"CONFIG_FILE={CONFIG_FILE}")
with open(CONFIG_FILE, 'r') as file:
    config = yaml.safe_load(file)

DATA_DIR: str = "data"
PROMPTS_DIR = os.path.join(DATA_DIR, "prompts")
METRICS_DIR = os.path.join(DATA_DIR, "metrics", config['general']['name'])
METRICS_PER_INFERENCE_DIR  = os.path.join(METRICS_DIR, "per_inference")
METRICS_PER_CHUNK_DIR  = os.path.join(METRICS_DIR, "per_chunk")
MODELS_DIR = os.path.join(DATA_DIR, "models", config['general']['name'])
DATASET_DIR = os.path.join(DATA_DIR, "dataset")
SCRIPTS_DIR: str = "scripts"
DIR_LIST = [DATA_DIR, PROMPTS_DIR, METRICS_DIR, MODELS_DIR, DATASET_DIR, METRICS_PER_INFERENCE_DIR, METRICS_PER_CHUNK_DIR]
TOKENIZER_DIR = 'llama2_tokenizer'

_ = list(map(lambda x: os.makedirs(x, exist_ok=True), DIR_LIST))

ENDPOINT_LIST_FPATH:str = os.path.join(MODELS_DIR, "endp

In [63]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#### Load the Config.yml file that contains information that is used across this benchmarking environment

In [64]:
config = load_config(CONFIG_FILE)
logger.info(json.dumps(config, indent=2))

[2024-01-23 19:28:28,545] p1345 {635462509.py:2} INFO - {
  "general": {
    "name": "mistral-7b-tgi-g5-v1",
    "model_name": "mistral7b"
  },
  "aws": {
    "region": "us-east-1"
  },
  "prompt": {
    "template_file": "prompt_template.txt",
    "all_prompts_file": "all_prompts.csv"
  },
  "datasets": [
    {
      "language": "en",
      "min_length_in_tokens": 1,
      "max_length_in_tokens": 500,
      "payload_file": "payload_{lang}_{min}-{max}.jsonl"
    },
    {
      "language": "en",
      "min_length_in_tokens": 500,
      "max_length_in_tokens": 1000,
      "payload_file": "payload_{lang}_{min}-{max}.jsonl"
    },
    {
      "language": "en",
      "min_length_in_tokens": 1000,
      "max_length_in_tokens": 2000,
      "payload_file": "payload_{lang}_{min}-{max}.jsonl"
    },
    {
      "language": "en",
      "min_length_in_tokens": 2000,
      "max_length_in_tokens": 3000,
      "payload_file": "payload_{lang}_{min}-{max}.jsonl"
    },
    {
      "language": "en",
    

In [65]:
date_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

### Access the deployed model endpoints from the endpoints.json file 

In [66]:
# read the list of deployed endpoints
endpoint_info_list = json.loads(Path(ENDPOINT_LIST_FPATH).read_text())
logger.info(f"found information for {len(endpoint_info_list)} endpoints")
logger.info(json.dumps(endpoint_info_list, indent=2))

[2024-01-23 19:28:35,125] p1345 {3756670252.py:3} INFO - found information for 1 endpoints
[2024-01-23 19:28:35,126] p1345 {3756670252.py:4} INFO - [
  {
    "experiment_name": "mistral-7b-g5-huggingface-pytorch-tgi-inference-2.0.1-tgi1.1.0",
    "endpoint": {
      "EndpointName": "lmistral7b-g5-2xlarge-1706037700",
      "EndpointArn": "arn:aws:sagemaker:us-east-1:015469603702:endpoint/lmistral7b-g5-2xlarge-1706037700",
      "EndpointConfigName": "lmistral7b-g5-2xlarge-1706037700",
      "ProductionVariants": [
        {
          "VariantName": "AllTraffic",
          "DeployedImages": [
            {
              "SpecifiedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04",
              "ResolvedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference@sha256:2739b630b95d8a95e6b4665e66d8243dd43b99c4fdb865feff13aab9c1da06eb",
              "ResolutionTime": "2024-01-23 19

In [67]:
# List down the endpoint names that have been deployed
endpoint_name_list = [e['endpoint']['EndpointName'] for e in endpoint_info_list]
logger.info(f"there are {len(endpoint_name_list)} deployed endpoint(s), endpoint_name_list->{endpoint_name_list}")

[2024-01-23 19:28:41,095] p1345 {1455142584.py:3} INFO - there are 1 deployed endpoint(s), endpoint_name_list->['lmistral7b-g5-2xlarge-1706037700']


### Creating predictor objects from the deployed endpoints

In [68]:
# create predictor objects

## create a sagemaker predictor for these endpoints
def create_predictor(endpoint_name: str) -> Optional[sagemaker.base_predictor.Predictor]:
    # Create a SageMaker Predictor object
    predictor = Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker.Session(),
        serializer=JSONSerializer()
    )
    return predictor

## Display the list of predictor objects that have been deployed ready for inferencing from
predictor_list: List = [create_predictor(ep) for ep in endpoint_name_list]
logger.info(predictor_list)

[2024-01-23 19:28:43,288] p1345 {3970465137.py:15} INFO - [<sagemaker.base_predictor.Predictor object at 0x7ff6ec6c9db0>]


### Creating functions to define and calculate metrics during the time of invocations

In [69]:
def safe_sum(l: List) -> Union[int, float]:
    return sum(filter(None, l))

def safe_div(n: Union[int, float], d: Union[int, float]) -> Optional[Union[int, float]]:
    return n/d if d else None

## Represents the function to calculate all of the metrics at the time of inference
def calculate_metrics(responses, chunk, elapsed_async, experiment_name, concurrency, payload_file) -> Dict:
    
    ## calculate errors based on the completion status of the inference prompt
    errors = [r for r in responses if r['completion'] is None]
    
    ## Calculate the difference as the successes 
    successes = len(chunk) - len(errors)
    
    ## Count all of the prompts token count during inference
    all_prompts_token_count = safe_sum([r['prompt_tokens'] for r in responses])
    prompt_token_throughput = round(all_prompts_token_count / elapsed_async, 2)
    prompt_token_count_mean = safe_div(all_prompts_token_count, successes)
    all_completions_token_count = safe_sum([r['completion_tokens'] for r in responses])
    completion_token_throughput = round(all_completions_token_count / elapsed_async, 2)
    completion_token_count_mean = safe_div(all_completions_token_count, successes)
    transactions_per_second = round(successes / elapsed_async, 2)
    transactions_per_minute = int(transactions_per_second * 60)
    
    ## calculate the latency mean utilizing the safe_sum function defined above
    latency_mean = safe_div(safe_sum([r['latency'] for r in responses]), successes)
    
    ## Function returns all these values at the time of the invocations
    return {
        'experiment_name': experiment_name,
        'concurrency': concurrency,
        'payload_file': payload_file,
        'errors': errors,
        'successes': successes,
        'error_rate': len(errors)/len(chunk),
        'all_prompts_token_count': all_prompts_token_count,
        'prompt_token_count_mean': prompt_token_count_mean,
        'prompt_token_throughput': prompt_token_throughput,
        'all_completions_token_count': all_completions_token_count,
        'completion_token_count_mean': completion_token_count_mean,
        'completion_token_throughput': completion_token_throughput,
        'transactions': len(chunk),
        'transactions_per_second': transactions_per_second,
        'transactions_per_minute': transactions_per_minute,
        'latency_mean': latency_mean
    }

### Set a blocker function and a series of asynchronous concurrent model prompt invocations

In [70]:
def set_metrics(endpoint_name=None,
                    prompt=None,
                    inference_params=None,
                    completion=None,
                    prompt_tokens=None,
                    completion_tokens=None,
                    latency=None) -> Dict:
    return dict(endpoint_name=endpoint_name,                
                prompt=prompt,
                **inference_params,
                completion=completion,
                prompt_tokens=prompt_tokens,
                completion_tokens=completion_tokens,
                latency=latency)

def get_inference(predictor, payload) -> Dict:
    
    smr_client = boto3.client("sagemaker-runtime")
    latency = 0

    try:
        prompt_tokens = count_tokens(payload['inputs'])
        logger.info(f"get_inference, endpoint={predictor.endpoint_name}, prompt_tokens={prompt_tokens}")

        # get inference
        st = time.perf_counter()        
        response = predictor.predict(payload)        
        latency = time.perf_counter() - st

        if isinstance(response, bytes):
            response = response.decode('utf-8')
        response_json = json.loads(response)
        if isinstance(response_json, list):
            response_json = response_json[0]

        completion = response_json.get("generated_text", "")
        completion_tokens = count_tokens(completion)

        # Set metrics and logging for both cases
        response = set_metrics(predictor.endpoint_name,
                               payload['inputs'],
                               payload['parameters'],
                               completion,
                               prompt_tokens,
                               completion_tokens,
                               latency)
        # logger.info(f"get_inference, done, endpoint={predictor.endpoint_name}, response={json.dumps(response, indent=2)}, latency={latency:.2f}")
        logger.info(f"get_inference, done, endpoint={predictor.endpoint_name}, completion_tokens={completion_tokens}, latency={latency:.2f}")
    except Exception as e:
        print(f"error occurred with {predictor.endpoint_name}, exception={str(e)}")
        response = set_metrics(predictor.endpoint_name,
                               payload['inputs'],
                               payload['parameters'],
                               None,
                               prompt_tokens,
                               None,
                               None)

    return response

### Setting a series of asynchronous functions to invoke and run inferences concurrently and asynchronously

In [72]:
## Represents a function to start invoking models in separate thread asynchronously for the blocker function
async def async_get_inference(predictor, payload: Dict) -> Dict:
    return await asyncio.to_thread(get_inference, predictor, payload)

## Gathers all of the tasks and sets of the concurrent calling of the asychronous invocations
async def async_get_all_inferences(predictor, payload_list: List) -> List:
    return await asyncio.gather(*[async_get_inference(predictor, payload) for payload in payload_list])

In [73]:
## This function runs the asychronous function series above together for different experiments and concurrency levels.
async def run_inferences(predictor: sagemaker.base_predictor.Predictor, chunk: List, experiment: Dict, concurrency: int, payload_file: str) -> Tuple[List, Dict]:
    logger.info(f"Processing chunk with concurrency={concurrency}")
    s = time.perf_counter()
    responses = await async_get_all_inferences(predictor, chunk)
    elapsed_async = time.perf_counter() - s

    # Add more metadata about this experiment
    for r in responses:
        r['experiment_name'] = experiment['name']
        r['concurrency'] = concurrency

    metrics = calculate_metrics(responses, chunk, elapsed_async, experiment['name'], concurrency, payload_file)
    return responses, metrics

In [74]:
## Function to create the predictors from the experiment we are iterating over
def create_predictor_for_experiment(experiment: str, config: Dict, endpoint_info_list: List) -> Optional[sagemaker.base_predictor.Predictor]:

    ## Here, we set the index and then iterate through the experiments
    e_idx = config['experiments'].index(experiment) + 1

    ## Iterate through the endpoint information to fetch the endpoint name
    ep_info = [e for e in endpoint_info_list if e['experiment_name'] == experiment['name']]
    if not ep_info:
        logger.error(f"endpoint for experiment={experiment['name']} not found, skipping")
        return None
    ep_name = ep_info[0]['endpoint']['EndpointName']
    logger.info(f"experiment={e_idx}, name={experiment['name']}, ep_name={ep_name}")

    # create a predictor from each endpoint in experiments
    return create_predictor(ep_name)

In [75]:
## Here, we will process combinations of concurrency levels, the payload files and then loop through the 
## different combinations to make payloads splitted in terms of the concurrency metric and how we can run 
## it and make inference

def create_payload_dict(jline: str, experiment: Dict) -> Dict:
    payload: Dict = json.loads(jline)
    if experiment.get('remove_truncate', False) is True:
        if payload['parameters'].get('truncate'):
            del payload['parameters']['truncate']
    return payload
    
    
def create_combinations(experiment: Dict) -> List[Tuple]:
    combinations_data = []

    # Repeat for each concurrency level
    combinations = list(itertools.product(experiment['concurrency_levels'], experiment['payload_files']))
    logger.info(f"there are {len(combinations)} combinations of {combinations} to run")

    for concurrency, payload_file in combinations:
        # Read the payload file
        fpath = os.path.join(PROMPTS_DIR, payload_file)
        payload_list = [create_payload_dict(jline, experiment) for jline in Path(fpath).read_text().splitlines()]
        logger.info(f"read {fpath}, contains {len(payload_list)} lines")      

        logger.info(f"creating combinations for concurrency={concurrency}, payload_file={payload_file}, payload_list length={len(payload_list)}")
        
        # check if we have enough element in the list to run at least the concurrency count numberof transactions..
        # for example if there are only 2 prompts and we want to run say 6 in parallel then take the first element (prompt)
        # and replicate it 6-2=4 times and add it to the original list
        n = concurrency
        
        if len(payload_list) < n:
            elements_to_add = n - len(payload_list)
            element_to_replicate = payload_list[0]
            # payload_list = payload_list.extend([element_to_replicate]*elements_to_add)
            payload_list.extend([element_to_replicate]*elements_to_add)
            
        # Split the original list into sublists which contain the number of requests we want to send concurrently        
        payload_list_splitted = [payload_list[i * n:(i + 1) * n] for i in range((len(payload_list) + n - 1) // n )]  
        
        for p in payload_list_splitted:
            if len(p) < n:
                elements_to_add = n - len(p)
                element_to_replicate = p[0]
                # p = p.extend([element_to_replicate]*elements_to_add)
                p.extend([element_to_replicate]*elements_to_add)
            

        # Only keep lists that have at least concurrency number of elements
        len_before = len(payload_list_splitted)
        payload_list_splitted = [p for p in payload_list_splitted if len(p) == concurrency]
        logger.info(f"after only retaining chunks of length {concurrency}, we have {len(payload_list_splitted)} chunks, previously we had {len_before} chunks")
        combinations_data.append((concurrency, payload_file, payload_list_splitted))
    logger.info(f"there are {len(combinations)} for {experiment}")
    return combinations_data

# process_combinations(experiment, predictor, PROMPTS_DIR)

In [76]:
# for each experiment
#   - for each endpoint and concurrency in an experiment

def clear_dir(dir_path: str):
    files = glob.glob(os.path.join(dir_path, "*"))
    for f in files:
        os.remove(f)

_ = list(map(clear_dir, [METRICS_PER_INFERENCE_DIR, METRICS_PER_CHUNK_DIR]))

num_experiments: int = len(config['experiments'])
for e_idx, experiment in enumerate(config['experiments']):
    e_idx += 1  # Increment experiment index
    # Call do_experiment function to create the predictor object
 
    predictor = create_predictor_for_experiment(experiment, config, endpoint_info_list)
    if predictor is None:
        logger.error(f"predictor could not be created for experiment={experiment}, moving to next...")
        continue

    # Process combinations of concurrency levels and payload files
    combination_data = create_combinations(experiment)

    for concurrency, payload_file, split_payload in combination_data:
        for chunk_index, chunk in enumerate(split_payload):
            logger.info(f"e_idx={e_idx}/{num_experiments}, chunk_index={chunk_index+1}/{len(split_payload)}")

            # Process each chunk and calculate metrics
            responses, metrics = await run_inferences(predictor, chunk, experiment, concurrency, payload_file)
            if metrics:
                #per_concurrency_level_response_metrics.append(metrics)
                fpath: str = os.path.join(METRICS_PER_CHUNK_DIR, f"{time.time()}.json")
                Path(fpath).write_text(json.dumps(metrics, indent=2))
            if responses:
                for r in responses:
                    fpath: str = os.path.join(METRICS_PER_INFERENCE_DIR, f"{time.time()}.json")
                    Path(fpath).write_text(json.dumps(r, indent=2))
            
            logger.info(f"completed processing chunk {chunk_index+1}/{len(split_payload)} with concurrency={concurrency}")

    logger.info(f"experiment={e_idx}/{num_experiments}, name={experiment['name']}, done")

[2024-01-23 19:29:31,082] p1345 {663335596.py:13} INFO - experiment=1, name=mistral-7b-g5-huggingface-pytorch-tgi-inference-2.0.1-tgi1.1.0, ep_name=lmistral7b-g5-2xlarge-1706037700
[2024-01-23 19:29:31,108] p1345 {1352403322.py:18} INFO - there are 25 combinations of [(1, 'payload_en_1-500.jsonl'), (1, 'payload_en_500-1000.jsonl'), (1, 'payload_en_1000-2000.jsonl'), (1, 'payload_en_2000-3000.jsonl'), (1, 'payload_en_3000-4000.jsonl'), (2, 'payload_en_1-500.jsonl'), (2, 'payload_en_500-1000.jsonl'), (2, 'payload_en_1000-2000.jsonl'), (2, 'payload_en_2000-3000.jsonl'), (2, 'payload_en_3000-4000.jsonl'), (4, 'payload_en_1-500.jsonl'), (4, 'payload_en_500-1000.jsonl'), (4, 'payload_en_1000-2000.jsonl'), (4, 'payload_en_2000-3000.jsonl'), (4, 'payload_en_3000-4000.jsonl'), (6, 'payload_en_1-500.jsonl'), (6, 'payload_en_500-1000.jsonl'), (6, 'payload_en_1000-2000.jsonl'), (6, 'payload_en_2000-3000.jsonl'), (6, 'payload_en_3000-4000.jsonl'), (8, 'payload_en_1-500.jsonl'), (8, 'payload_en_500-

In [77]:
# read all per chunk files
json_list: List[Dict] = [json.loads(Path(f_name).read_text()) for f_name in glob.glob(os.path.join(METRICS_PER_INFERENCE_DIR, "*.json"))]
df_responses = pd.DataFrame(json_list)
logger.info(f"created dataframe of shape {df_responses.shape} from all responses")
df_responses.head()


[2024-01-23 19:49:14,664] p1345 {3838589563.py:4} INFO - created dataframe of shape (586, 14) from all responses


,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,completion_tokens,latency,experiment_name,concurrency
0,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,2321,\n```\n\n```\n\nQuestion: Passage:\nBiggest We...,2321,100,4.599911,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
1,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3436,Where Are You? I'm Here is earlier than Patna...,3436,33,5.953868,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,8
2,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3655,Yes.,3655,4,6.470846,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,8
3,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,2691,\n\n\n[INST] <<SYS>>\nYou are an assistant for...,2691,101,4.817750,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
4,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3271,\n\n\n[/INST]\nAnswer:\nThe Owl and the Pussy-...,3271,103,7.673864,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,4


In [78]:
# read all per inference files
json_list: List[Dict] = [json.loads(Path(f_name).read_text()) for f_name in glob.glob(os.path.join(METRICS_PER_INFERENCE_DIR, "*.json"))]
df_responses = pd.DataFrame(json_list)
logger.info(f"created dataframe of shape {df_responses.shape} from all inference responses")
df_responses.head()


[2024-01-23 19:49:15,360] p1345 {1971204328.py:4} INFO - created dataframe of shape (586, 14) from all inference responses


,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,completion_tokens,latency,experiment_name,concurrency
0,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,2321,\n```\n\n```\n\nQuestion: Passage:\nBiggest We...,2321,100,4.599911,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
1,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3436,Where Are You? I'm Here is earlier than Patna...,3436,33,5.953868,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,8
2,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3655,Yes.,3655,4,6.470846,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,8
3,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,2691,\n\n\n[INST] <<SYS>>\nYou are an assistant for...,2691,101,4.817750,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
4,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3271,\n\n\n[/INST]\nAnswer:\nThe Owl and the Pussy-...,3271,103,7.673864,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,4


In [79]:
df_responses[df_responses.endpoint_name.str.contains("inf2")]

,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,completion_tokens,latency,experiment_name,concurrency


In [80]:
# read all per inference files
json_list: List[Dict] = [json.loads(Path(f_name).read_text()) for f_name in glob.glob(os.path.join(METRICS_PER_CHUNK_DIR, "*.json"))]
df_metrics = pd.DataFrame(json_list)
logger.info(f"created dataframe of shape {df_metrics.shape} from all chunk responses")
df_metrics.head()

[2024-01-23 19:49:15,848] p1345 {976317414.py:4} INFO - created dataframe of shape (227, 16) from all chunk responses


,experiment_name,concurrency,payload_file,errors,successes,error_rate,all_prompts_token_count,prompt_token_count_mean,prompt_token_throughput,all_completions_token_count,completion_token_count_mean,completion_token_throughput,transactions,transactions_per_second,transactions_per_minute,latency_mean
0,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,8,payload_en_2000-3000.jsonl,[],8,0.0,20226,2528.25,1468.72,718,89.75,52.14,8,0.58,34,7.923026
1,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,2,payload_en_3000-4000.jsonl,[],2,0.0,6626,3313.00,1082.04,105,52.50,17.15,2,0.33,19,4.077796
2,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,6,payload_en_1000-2000.jsonl,[],6,0.0,9369,1561.50,1479.16,351,58.50,55.42,6,0.95,57,4.601383
3,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,4,payload_en_3000-4000.jsonl,[],4,0.0,13239,3309.75,1679.03,213,53.25,27.01,4,0.51,30,5.716153
4,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,4,payload_en_3000-4000.jsonl,[],4,0.0,14395,3598.75,1566.69,403,100.75,43.86,4,0.44,26,8.499557


In [81]:
df_endpoints = pd.json_normalize(endpoint_info_list)
df_endpoints['instance_type'] = df_endpoints['endpoint_config.ProductionVariants'].map(lambda x: x[0]['InstanceType'])
df_endpoints
cols_for_env = [c for c in df_endpoints.columns if 'Environment' in c]
print(cols_for_env)
cols_of_interest = ['experiment_name', 
                    'instance_type',
                    'endpoint.EndpointName',
                    'model_config.ModelName',
                    'model_config.PrimaryContainer.Image',   
                    'model_config.PrimaryContainer.ModelDataSource.S3DataSource.S3Uri']
cols_of_interest.extend(cols_for_env)

df_endpoints = df_endpoints[cols_of_interest]
df_endpoints = df_endpoints[cols_of_interest]
cols_of_interest_renamed = [c.split('.')[-1] for c in cols_of_interest]
df_endpoints.columns = cols_of_interest_renamed

# Check if 'experiment_name' column exists in both DataFrames
print("Columns in df_responses:", df_responses.columns)
print("Columns in df_endpoints:", df_endpoints.columns)

# Merge operation
df_results = pd.merge(left=df_responses, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')

# Inspect the result
df_results.head()

['model_config.PrimaryContainer.Environment.ENDPOINT_SERVER_TIMEOUT', 'model_config.PrimaryContainer.Environment.HF_MODEL_ID', 'model_config.PrimaryContainer.Environment.MAX_BATCH_PREFILL_TOKENS', 'model_config.PrimaryContainer.Environment.MAX_INPUT_LENGTH', 'model_config.PrimaryContainer.Environment.MAX_TOTAL_TOKENS', 'model_config.PrimaryContainer.Environment.MODEL_CACHE_ROOT', 'model_config.PrimaryContainer.Environment.SAGEMAKER_ENV', 'model_config.PrimaryContainer.Environment.SAGEMAKER_MODEL_SERVER_WORKERS', 'model_config.PrimaryContainer.Environment.SAGEMAKER_PROGRAM', 'model_config.PrimaryContainer.Environment.SM_NUM_GPUS']
Columns in df_responses: Index(['endpoint_name', 'prompt', 'do_sample', 'temperature', 'top_p', 'top_k',
       'max_new_tokens', 'truncate', 'completion', 'prompt_tokens',
       'completion_tokens', 'latency', 'experiment_name', 'concurrency'],
      dtype='object')
Columns in df_endpoints: Index(['experiment_name', 'instance_type', 'EndpointName', 'ModelNam

,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,...,ENDPOINT_SERVER_TIMEOUT,HF_MODEL_ID,MAX_BATCH_PREFILL_TOKENS,MAX_INPUT_LENGTH,MAX_TOTAL_TOKENS,MODEL_CACHE_ROOT,SAGEMAKER_ENV,SAGEMAKER_MODEL_SERVER_WORKERS,SAGEMAKER_PROGRAM,SM_NUM_GPUS
0,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,2321,\n```\n\n```\n\nQuestion: Passage:\nBiggest We...,2321,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
1,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3436,Where Are You? I'm Here is earlier than Patna...,3436,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
2,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3655,Yes.,3655,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
3,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,2691,\n\n\n[INST] <<SYS>>\nYou are an assistant for...,2691,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
4,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3271,\n\n\n[/INST]\nAnswer:\nThe Owl and the Pussy-...,3271,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1


In [82]:
df_results = pd.merge(left=df_responses, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')
df_results.head()

,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,...,ENDPOINT_SERVER_TIMEOUT,HF_MODEL_ID,MAX_BATCH_PREFILL_TOKENS,MAX_INPUT_LENGTH,MAX_TOTAL_TOKENS,MODEL_CACHE_ROOT,SAGEMAKER_ENV,SAGEMAKER_MODEL_SERVER_WORKERS,SAGEMAKER_PROGRAM,SM_NUM_GPUS
0,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,2321,\n```\n\n```\n\nQuestion: Passage:\nBiggest We...,2321,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
1,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3436,Where Are You? I'm Here is earlier than Patna...,3436,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
2,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3655,Yes.,3655,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
3,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,2691,\n\n\n[INST] <<SYS>>\nYou are an assistant for...,2691,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
4,lmistral7b-g5-2xlarge-1706037700,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.7,0.92,120,100,3271,\n\n\n[/INST]\nAnswer:\nThe Owl and the Pussy-...,3271,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1


In [83]:
fpath: str = os.path.join(METRICS_DIR, config['results']['per_inference_request_file']).format(datetime=date_time)
df_results.to_csv(fpath, index=False)
logger.info(f"saved results dataframe of shape={df_results.shape} in {fpath}")

[2024-01-23 19:49:16,263] p1345 {3435721265.py:3} INFO - saved results dataframe of shape=(586, 29) in data/metrics/mistral-7b-tgi-g5-v1/per_inference_request_results.csv


In [84]:
df_metrics = pd.merge(left=df_metrics, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')
df_metrics.head()
fpath: str = os.path.join(METRICS_DIR, config['results']['all_metrics_file']).format(datetime=date_time)
df_metrics.to_csv(fpath, index=False)
logger.info(f"saved metrics results dataframe of shape={df_metrics.shape} in {fpath}")

[2024-01-23 19:49:16,326] p1345 {3880536130.py:5} INFO - saved metrics results dataframe of shape=(227, 31) in data/metrics/mistral-7b-tgi-g5-v1/all_metrics.csv
